In [34]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import os
from math import * 

print("The current directory is: ")
print(os.getcwd())
if not os.getcwd().endswith("Abaqus-Hardening-Seq-2-Seq-Project"):
    # Move up two directories
    path_parent = os.path.dirname(os.getcwd())
    os.chdir(path_parent)
    path_parent = os.path.dirname(os.getcwd())
    os.chdir(path_parent)
print("The current directory is: ")
print(os.getcwd())

from modules.IO import *
from modules.calculation import *

The current directory is: 
c:\Users\springnuance\Desktop\Abaqus-Hardening-Seq-2-Seq-Project


In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
truePlasticStrainConfig = pd.read_excel(f"configs/truePlasticStrain_config.xlsx",engine="openpyxl")
ranges_and_increments = []

for index, row in truePlasticStrainConfig.iterrows():
    ranges_and_increments.append((row['strainStart'], row['strainEnd'], row['strainStep']))
    
truePlasticStrain = np.array([])

# Iterate through each range and increment
for i, (start, end, step) in enumerate(ranges_and_increments):
    # Skip the start value for all ranges after the first one
    if i > 0:
        start += step
    # Create numpy array for range
    strain_range = np.arange(start, end+1e-12, step)
    strain_range = np.around(strain_range, decimals=8)
    # Append strain_range to strain_array
    truePlasticStrain = np.concatenate((truePlasticStrain, strain_range))

print(len(truePlasticStrain))
print(truePlasticStrain)
# np.save(f"configs/truePlasticStrain_{hardeningLaw}.npy", truePlasticStrain)

100
[0.00e+00 5.00e-04 1.00e-03 2.00e-03 3.00e-03 4.00e-03 5.00e-03 6.00e-03
 7.00e-03 8.00e-03 9.00e-03 1.00e-02 1.50e-02 2.00e-02 2.50e-02 3.00e-02
 3.50e-02 4.00e-02 4.50e-02 5.00e-02 5.50e-02 6.00e-02 6.50e-02 7.00e-02
 7.50e-02 8.00e-02 8.50e-02 9.00e-02 9.50e-02 1.00e-01 1.05e-01 1.10e-01
 1.15e-01 1.20e-01 1.25e-01 1.30e-01 1.35e-01 1.40e-01 1.45e-01 1.50e-01
 1.55e-01 1.60e-01 1.65e-01 1.70e-01 1.75e-01 1.80e-01 1.85e-01 1.90e-01
 1.95e-01 2.00e-01 2.10e-01 2.20e-01 2.30e-01 2.40e-01 2.50e-01 2.60e-01
 2.70e-01 2.80e-01 2.90e-01 3.00e-01 3.10e-01 3.20e-01 3.30e-01 3.40e-01
 3.50e-01 3.60e-01 3.70e-01 3.80e-01 3.90e-01 4.00e-01 4.10e-01 4.20e-01
 4.30e-01 4.40e-01 4.50e-01 4.60e-01 4.70e-01 4.80e-01 4.90e-01 5.00e-01
 5.50e-01 6.00e-01 6.50e-01 7.00e-01 7.50e-01 8.00e-01 8.50e-01 9.00e-01
 9.50e-01 1.00e+00 1.20e+00 1.40e+00 1.60e+00 1.80e+00 2.00e+00 2.20e+00
 2.40e+00 2.60e+00 2.80e+00 3.00e+00]


In [37]:
import numpy as np

#================ defining the hardening laws =============================

def Swift(c1,c2,c3, truePlasticStrain):
    trueStress = c1 * (c2 + truePlasticStrain) ** c3
    return trueStress

def Voce(c1,c2,c3,truePlasticStrain):
    trueStress = c1 + c2 * (1- np.exp(-c3 * truePlasticStrain))
    return trueStress

def SwiftVoce(c1,c2,c3,c4,c5,c6,c7,truePlasticStrain):
    trueStressSwift = Swift(c2,c3,c4,truePlasticStrain)
    trueStressVoce = Voce(c5,c6,c7,truePlasticStrain)
    trueStress = c1 * trueStressSwift + (1 - c1) * trueStressVoce
    return trueStress

def calculate_flowCurve(parameters, hardeningLaw, truePlasticStrain):
    # We assume that parameters is a dictionary
    if hardeningLaw == "Swift":
        c1, c2, c3 = parameters["c1"], parameters["c2"], parameters["c3"]
        trueStress = Swift(c1, c2, c3, truePlasticStrain)
    elif hardeningLaw == "Voce":
        c1, c2, c3 = parameters["c1"], parameters["c2"], parameters["c3"]
        trueStress = Voce(c1, c2, c3, truePlasticStrain)
    elif hardeningLaw == "SwiftVoce":
        c1, c2, c3, c4, c5, c6, c7 = parameters["c1"], parameters["c2"], parameters["c3"], parameters["c4"], parameters["c5"], parameters["c6"], parameters["c7"]
        trueStress = SwiftVoce(c1, c2, c3, c4, c5, c6, c7, truePlasticStrain)
    return trueStress
        

In [38]:
material = "DP1000_25C"
resultPath_SV = f"results_SV/{material}"
resultPath = f"results/{material}"
geometries = ["CHD6", "NDBR6", "NDBR20", "NDBR50", "SH"]
stages = ["initial", "iteration"]
#plt.figure(figsize=(10, 10))

print(len(truePlasticStrain))
for geometry in geometries:
    for stage in stages:
        FD_Curves_smooth = np.load(f"{resultPath_SV}/{geometry}/{stage}/common/FD_Curves_smooth.npy", allow_pickle=True).tolist()
        FD_Curves_smooth_noSV = []
        for key, value in FD_Curves_smooth.items():
            FD_Curves_smooth_noSV.append(value)
        np.save(f"{resultPath}/{geometry}/{stage}/common/FD_Curves_smooth.npy", FD_Curves_smooth_noSV)
        
        FD_Curves_unsmooth = np.load(f"{resultPath_SV}/{geometry}/{stage}/common/FD_Curves_unsmooth.npy", allow_pickle=True).tolist()
        FD_Curves_unsmooth_noSV = []
        for key, value in FD_Curves_unsmooth.items():
            FD_Curves_unsmooth_noSV.append(value)
        np.save(f"{resultPath}/{geometry}/{stage}/common/FD_Curves_unsmooth.npy", FD_Curves_unsmooth_noSV)

        flowCurves = np.load(f"{resultPath_SV}/{geometry}/{stage}/common/flowCurves.npy", allow_pickle=True).tolist()
        flowCurves_noSV = []
        for key, value in flowCurves.items():
            trueStress = calculate_flowCurve(dict(key), "SwiftVoce", truePlasticStrain)
            new_value = {"strain": truePlasticStrain, "stress": trueStress}
            flowCurves_noSV.append(new_value)
            #plt.plot(truePlasticStrain, trueStress, color="orange", alpha=0.05)
        np.save(f"{resultPath}/{geometry}/{stage}/common/flowCurves.npy", flowCurves_noSV)


100


In [39]:
material = "DP1000_400C"
resultPath_SV = f"results_SV/{material}"
resultPath = f"results/{material}"
geometries = ["CHD6", "NDBR6", "NDBR50"]
stages = ["initial", "iteration"]
#plt.figure(figsize=(10, 10))

print(len(truePlasticStrain))
for geometry in geometries:
    for stage in stages:
        FD_Curves_smooth = np.load(f"{resultPath_SV}/{geometry}/{stage}/common/FD_Curves_smooth.npy", allow_pickle=True).tolist()
        FD_Curves_smooth_noSV = []
        for key, value in FD_Curves_smooth.items():
            FD_Curves_smooth_noSV.append(value)
        np.save(f"{resultPath}/{geometry}/{stage}/common/FD_Curves_smooth.npy", FD_Curves_smooth_noSV)
        
        FD_Curves_unsmooth = np.load(f"{resultPath_SV}/{geometry}/{stage}/common/FD_Curves_unsmooth.npy", allow_pickle=True).tolist()
        FD_Curves_unsmooth_noSV = []
        for key, value in FD_Curves_unsmooth.items():
            FD_Curves_unsmooth_noSV.append(value)
        np.save(f"{resultPath}/{geometry}/{stage}/common/FD_Curves_unsmooth.npy", FD_Curves_unsmooth_noSV)

        flowCurves = np.load(f"{resultPath_SV}/{geometry}/{stage}/common/flowCurves.npy", allow_pickle=True).tolist()
        flowCurves_noSV = []
        for key, value in flowCurves.items():
            trueStress = calculate_flowCurve(dict(key), "SwiftVoce", truePlasticStrain)
            new_value = {"strain": truePlasticStrain, "stress": trueStress}
            flowCurves_noSV.append(new_value)
            #plt.plot(truePlasticStrain, trueStress, color="orange", alpha=0.05)
        np.save(f"{resultPath}/{geometry}/{stage}/common/flowCurves.npy", flowCurves_noSV)

100


In [40]:
material = "QP1000_25C"
resultPath_SV = f"results_SV/{material}"
resultPath = f"results/{material}"
geometries = ["CHD6", "NDBR6", "NDBR25"]
stages = ["initial", "iteration"]
#plt.figure(figsize=(10, 10))

print(len(truePlasticStrain))
for geometry in geometries:
    for stage in stages:
        FD_Curves_smooth = np.load(f"{resultPath_SV}/{geometry}/{stage}/common/FD_Curves_smooth.npy", allow_pickle=True).tolist()
        FD_Curves_smooth_noSV = []
        for key, value in FD_Curves_smooth.items():
            FD_Curves_smooth_noSV.append(value)
        np.save(f"{resultPath}/{geometry}/{stage}/common/FD_Curves_smooth.npy", FD_Curves_smooth_noSV)
        
        FD_Curves_unsmooth = np.load(f"{resultPath_SV}/{geometry}/{stage}/common/FD_Curves_unsmooth.npy", allow_pickle=True).tolist()
        FD_Curves_unsmooth_noSV = []
        for key, value in FD_Curves_unsmooth.items():
            FD_Curves_unsmooth_noSV.append(value)
        np.save(f"{resultPath}/{geometry}/{stage}/common/FD_Curves_unsmooth.npy", FD_Curves_unsmooth_noSV)

        flowCurves = np.load(f"{resultPath_SV}/{geometry}/{stage}/common/flowCurves.npy", allow_pickle=True).tolist()
        flowCurves_noSV = []
        for key, value in flowCurves.items():
            trueStress = calculate_flowCurve(dict(key), "SwiftVoce", truePlasticStrain)
            new_value = {"strain": truePlasticStrain, "stress": trueStress}
            flowCurves_noSV.append(new_value)
            #plt.plot(truePlasticStrain, trueStress, color="orange", alpha=0.05)
        np.save(f"{resultPath}/{geometry}/{stage}/common/flowCurves.npy", flowCurves_noSV)


100
